In [10]:
%load_ext autoreload
%reload_ext autoreload

%autoreload 2
%matplotlib inline
import tensorflow
from tensorflow.compat.v1.keras.backend import get_session
tensorflow.compat.v1.disable_v2_behavior()
import kerasAC 
from kerasAC.generators.tiledb_predict_generator import *
from kerasAC.tiledb_config import *
import tiledb 
from scipy.special import softmax
from kerasAC.interpret.deepshap import * 
from kerasAC.interpret.profile_shap import * 
from kerasAC.vis import * 

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [5]:
gen=TiledbPredictGenerator(ref_fasta="/mnt/data/GRCh38_no_alt_analysis_set_GCA_000001405.15.fasta",
                    batch_size=1,
                    bed_regions_summit_center=True,
                    bed_regions="/srv/scratch/annashch/bias_correction/enzymatic_bias/profile_model_of_bias/dnase/k562.chr1.bed",
                    tdb_array="/srv/scratch/annashch/bias_correction/enzymatic_bias/tiledb_inputs/merged.SRR1565781.SRR1565782",
                    chrom_sizes="/users/annashch/hg38.chrom.sizes",
                    tdb_input_flank=[673],
                    tdb_partition_attribute_for_upsample=[None],
                    tdb_input_source_attribute=["seq"],
                    tdb_partition_thresh_for_upsample=None,
                    tdb_input_aggregation=[None],
                    tdb_input_transformation=[None],
                    tdb_output_source_attribute=["bigwig_track","bigwig_track"],
                    tdb_output_flank=[500,500],
                    tdb_output_aggregation=[None,"sum"],
                    tdb_output_transformation=[None,"log"],
                    num_inputs=1,
                    num_outputs=2,
                    upsample_ratio=None,
                    tasks=['SRR1565781.SRR1565782'],
                    tdb_ambig_attribute="ambig_peak",
                    tdb_config=get_default_config(),
                    tdb_ctx=tiledb.Ctx(config=get_default_config()),
                    num_threads=1)
                    

opening:/srv/scratch/annashch/bias_correction/enzymatic_bias/tiledb_inputs/merged.SRR1565781.SRR1565782 for reading...
success!
got indices for used chroms
identified task indices:[0]
loaded bed regions
created generator
created predict generator


In [7]:
#load the model! 
from keras.models import load_model
from keras.utils.generic_utils import get_custom_objects
from kerasAC.metrics import * 
from kerasAC.custom_losses import * 
custom_objects={"recall":recall,
                    "sensitivity":recall,
                    "specificity":specificity,
                    "fpr":fpr,
                    "fnr":fnr,
                    "precision":precision,
                    "f1":f1,
                    "ambig_binary_crossentropy":ambig_binary_crossentropy,
                    "ambig_mean_absolute_error":ambig_mean_absolute_error,
                    "ambig_mean_squared_error":ambig_mean_squared_error,
                    "MultichannelMultinomialNLL":MultichannelMultinomialNLL}
get_custom_objects().update(custom_objects)


In [13]:
model=load_model("/srv/scratch/annashch/bias_correction/enzymatic_bias/profile_model_of_bias/dnase/bias.dnase.0.hdf5")
model_wrapper=(model.input, model.layers[-1].output)
count_explainer=shap.DeepExplainer(model_wrapper,data=create_background,combine_mult_and_diffref=combine_mult_and_diffref_1d)
prof_explainer = create_explainer(model)
all_coords=[] 
all_profile_explanations=[] 
all_count_explanations=[] 
for i in range(len(gen)): 
    X,y,coords=gen[i]
    coords=coords[0]
    profile_explanations=prof_explainer(X[0],None,None)
    count_explanations=np.squeeze(count_explainer.shap_values(X)[0])
    all_coords.append(coords)
    all_profile_explanations.append(profile_explanations)
    all_count_explanations.append(count_explanations)
    if i%50==0: 
        print(str(i))
        

here!
here!


0
50
100
150
200
250
300
350
400
450
500
550
600
650
700
750
800
850
900
950
1000
1050
1100
1150
1200
1250
1300
1350
1400
1450
1500
1550
1600
1650
1700
1750
1800
1850
1900
1950
2000
2050
2100
2150
2200
2250
2300
2350
2400
2450
2500
2550
2600
2650
2700
2750
2800
2850
2900
2950
3000
3050
3100
3150
3200
3250
3300
3350
3400
3450
3500
3550
3600
3650
3700
3750
3800
3850
3900
3950
4000
4050
4100
4150
4200
4250
4300
4350
4400
4450
4500
4550
4600
4650
4700
4750
4800
4850
4900
4950
5000
5050
5100
5150
5200
5250
5300
5350
5400
5450
5500
5550
5600
5650
5700
5750
5800
5850
5900
5950
6000
6050
6100
6150
6200
6250
6300
6350
6400
6450
6500
6550
6600
6650
6700
6750
6800
6850
6900
6950
7000
7050
7100
7150
7200
7250
7300
7350
7400
7450
7500
7550
7600
7650
7700
7750
7800
7850
7900
7950
8000
8050
8100
8150
8200
8250
8300
8350
8400
8450
8500
8550
8600
8650
8700
8750
8800
8850
8900
8950
9000
9050
9100
9150
9200
9250
9300
9350
9400
9450
9500
9550
9600
9650
9700
9750
9800
9850
9900
9950
10000
10050
10100
10150

KeyboardInterrupt: 

In [14]:
import pickle
outputs={} 
outputs['coords']=all_coords
outputs['shap_profile']=all_profile_explanations
outputs['shap_counts']=all_count_explanations
pickle.dump( outputs,open( "k562.deepshap.bpnet.p", "wb" ) )
